# Making slides with the Assistants API and DALL-E3

This notebook illustrates the use of the new [**Assistants API**](https://platform.openai.com/docs/assistants/overview) and [**DALL-E3**](https://platform.openai.com/docs/guides/images?context=node) in crafting informative and visually appealing slides. <br>
Creating engaging slides is a pivotal aspect of many jobs, but it can be a laborious and time-consuming task. Additionally, extracting insights from data and articulating them effectively on slides can be challenging. <br><br> This cookbook recipe will demonstrate how you can utilize the new Assistants API, with GPT-Vision, and DALL-E3 to create slides for you without you having to touch Microsoft PowerPoint or Google Slides, saving you valuable time and effort!

## 0. Setup

In [1]:
from IPython.display import display, Image
from openai import OpenAI
import pandas as pd
import json
import io
from PIL import Image
import requests


client = OpenAI()

#Lets import some helper functions for assistants from https://cookbook.openai.com/examples/assistants_api_overview_python
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def submit_message(assistant_id, thread, user_message,file_ids=None):
    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }
    if file_ids:
        params['file_ids']=file_ids

    client.beta.threads.messages.create(
        **params
)
    return client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)


## 1. Creating the content

In this recipe, we will be creating a brief fictional presentation for the quarterly financial review of our company, NotReal Corporation. In the presentation, we want to highlight some key trends we are seeing that are affecting the profitability of our company.<br> Let's say we have the some financial data at our disposal. Let's load in the data, and take a look...

In [2]:
financial_data_path = 'data/NotRealCorp_financial_data.json'
financial_data = pd.read_json(financial_data_path)
financial_data.head(5)


,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


As you can see, this data has quarterly revenue, costs and customer data across different distribution channels. Let's create an Assistant
that can act as a personal analyst and make a nice visualization for our PowerPoint!

First, we need to upload our file so our Assistant can access it.

In [3]:
file = client.files.create(
  file=open('data/NotRealCorp_financial_data.json',"rb"),
  purpose='assistants',
)


Now, we're ready to create our Assistant. We can instruct our assistant to act as a data scientist, and take any queries we give it and run the necessary code to output the proper data visualization. We can also turn on the tool of Code Interpreter, so our Assistant will be able to code. Finally, we can specifiy any files we want to use, which in this case is just the `financial_data` file we created above.

In [4]:
assistant = client.beta.assistants.create(
  instructions="You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)


Let's create a thread now, and as our first request ask the Assistant to calculate quarterly profits, and then plot the profits by distribution channel over time. The assistant will automatically calculate the profit for each quarter, and also create a new column combining quarter and year, without us having to ask for that directly. We can also specify the colors of each line.

In [5]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Calculate profit (revenue minus cost) by quarter and year, and visualize as a line plot across the distribution channels, where the colors of the lines are green, light red, and light blue",
      "file_ids": [file.id]
    }
  ]
)


No we can execute the run of our thread

In [6]:

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


We can now start a loop that will check if the image has been created. Note: This may take a few minutes

In [7]:
messages = client.beta.threads.messages.list(thread_id=thread.id)


In [8]:
import time

while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        #See if image has been created
        messages.data[0].content[0].image_file
        print('Plot created!')
        break
    except:
        time.sleep(10)
        print('Assistant still working...')


Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Plot created!


Let's see the messages the Assistant added.

In [9]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]


[MessageContentImageFile(image_file=ImageFile(file_id='file-D9tKO1cYsQZRIOevecE8PLxQ'), type='image_file'),
 MessageContentText(text=Text(annotations=[], value="With the profit calculated for each quarter, year, and distribution channel, the next step is to visualize this data as a line plot. We'll plot profit against time (combination of year and quarter) for each distribution channel and use the specified colors for the lines: green, light red, and light blue."), type='text'),
 MessageContentText(text=Text(annotations=[], value="The DataFrame has now been reconstructed with columns for the year, quarter, distribution channel, revenue, costs, customer count, and a time column that combines year and quarter. With this structure in place, we can now proceed to calculate the profit by quarter and year and then visualize it across the different distribution channels as requested.\n\nWe will calculate the profit by subtracting the costs from the revenue for each row, then aggregate this by

We can see that the last message (latest message is shown first) from the assistant contains the image file we are looking for.  We can write a quick helper function to convert our output file to a png

In [13]:
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)


In [16]:
plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "../images/NotRealCorp_chart.png"
convert_file_to_png(plot_file_id,image_path)

#Upload
plot_file = client.files.create(
  file=open(image_path, "rb"),
  purpose='assistants'
)



Let's load in the plot!

![Financial data plot](../images/NotRealCorp_chart.png)

Nice! So, with just one sentence, we were able to have our assistant use code interpreter to
calculate the profitability, and graph the three lineplots of the various distribution channels.<br><br>
Now we have a nice visual for our slide, but we want some insights to go along with it. Let's use GPT-Vision to help us generate those.

## 2. Generating insights

To get insights from our image, we simply need to add a new message to our thread. Our Assistant will know to use GPT-Vision, and can give us some concise takeaways from the visual provided.

In [15]:
submit_message(assistant.id,thread,"Give me two medium length sentences (~20-30 words per sentence) of the \
      most important insights from the plot you just created.\
             These will be used for a slide deck, and they should be about the\
                     'so what' behind the data."
)


Run(id='run_ZYc98irhdOZbduQsJQ9A1tHJ', assistant_id='asst_yjGxM3J9nrccJy8S40V0wgr8', cancelled_at=None, completed_at=None, created_at=1701805023, expires_at=1701805623, failed_at=None, file_ids=['file-pPIOhXgzBb0i3jskv2BZSLL9'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_bWFhNOorgF64jvhVQERECdSz', tools=[ToolAssistantToolsCode(type='code_interpreter')])

Now, once the run has completed, we can get the latest message

In [17]:
# Hard coded wait for a response, as the assistant may iterate on the bullets.
time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)


The line plot reveals that Online Sales consistently generates the highest profit across quarters, highlighting its effectiveness as a distribution channel. Direct Sales shows the least profit, indicating a potential area for strategic reassessment and improvement.


Cool! Now we have some two insights to accompany our plot. Now let's get a compelling title for the slide.

In [18]:
submit_message(assistant.id,thread,"Given the plot and bullet points you created,\
 come up with a very brief title for a slide. It should reflect just the main insights you came up with."
)


Run(id='run_5mxOWT6XgCNxz006DSAApacH', assistant_id='asst_yjGxM3J9nrccJy8S40V0wgr8', cancelled_at=None, completed_at=None, created_at=1701805214, expires_at=1701805814, failed_at=None, file_ids=['file-pPIOhXgzBb0i3jskv2BZSLL9'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_bWFhNOorgF64jvhVQERECdSz', tools=[ToolAssistantToolsCode(type='code_interpreter')])

And the title is

In [19]:
#Wait as assistant may take a few steps
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)


"Maximizing Profits: Online Sales Lead While Direct Sales Lag Behind"


## 3. DALL-E3 title image

Nice, now we have a title, a plot and two bullet points. We're almost ready to put this all on a slide, but as a final step, let's have DALL-E3 come up with an image to use as the title slide of the presentation. <br><br>
*Note:* DALL-E3 is not yet available within the assistants API but is coming soon! <br> <br>
We'll feed in a brief description of our company (NotRealCorp) and have DALL-E3 do the rest!

In [20]:
company_summary = "NotReal Corp is a prominent hardware company that manufactures and sells processors, graphics cards and other essential computer hardware."


In [21]:
response = client.images.generate(
  model='dall-e-3',
  prompt=f"given this company summary {company_summary}, create an inspirational \
    photo showing the growth and path forward. This will be used at a quarterly\
       financial planning meeting",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url


Cool, now we can add this image to our thread. First, we can save the image locally, then upload it to the assistants API using the `File` upload endpoint. Let's also take a look at our image

In [24]:
dalle_img_path = '../images/dalle_image.png'
img = requests.get(image_url)

#Save locally
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

#Upload
dalle_file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)


  
![Image](../images/dalle_image.png)


## 4. Creating the slides

We now have all the content we need to create the slides. While we could simply add a message asking for slides, but let's instead give the assistant a slide template, using the `python-pptx` library, to use. This will ensure we get a deck in the style we want. See the `Extensions` section at the end of the notebook for notes on creating the template.

In [ ]:
title_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Add image to the left side of the slide with a margin at the top and bottom
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text box positioned higher
left = prs.slide_width * 3/5
top = Inches(2)
width = prs.slide_width * 2/5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(38)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add subtitle text box
left = prs.slide_width * 3/5
top = Inches(3)
width = prs.slide_width * 2/5
height = Inches(1)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_p = subtitle_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(22)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
"""

data_vis_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Define placeholders
image_path = data_vis_img
title_text = "Maximizing Profits: The Dominance of Online Sales & Direct Sales Optimization"
bullet_points = "• Online Sales consistently lead in profitability across quarters, indicating a strong digital market presence.\n• Direct Sales show fluctuations, suggesting variable performance and the need for targeted improvements in that channel."

# Add image placeholder on the left side of the slide
left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text spanning the whole width
left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add hardcoded "Key Insights" text and bullet points
left = prs.slide_width * 2/3
top = Inches(1.5)
width = prs.slide_width * 1/3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "Key Insights:"
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()


bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""



Let's set a few quick variables for our slides. We want the company name, NotRealCorp, to be on the title slide, and the title of the presentation should 'Quartlerly financial planning metting, Q3, 2023'.

In [ ]:
title_text = "NotRealCorp"
subtitle_text = "Quarterly financial planning meeting, Q3 2023"


Now we can have our assistant create the slide. The assistant will automatically pull the bullet points and titles it made earlier into the slides. Just for 

Now, we have a PPTX file saved with all of our created content!. <br>
We don't have a `seed` parameter yet in the Assistants API, so the DALL-E3 image and wordings will be slightly different from what you see when you run this notebook, due to the non-deterministic quality of LLMs, but the outputs should be directionally the same.

Let's look at the screenshots of the .pptx we just created using JUST the assistants API and DALL-E3, we have the following:

And for the data slide, we have:

Woo! While these slides could use some small formatting tweaks, we have made some great content, using just the Assistants API, GPT-4, GPT-Vision, and DALL-E3. Hopefully this helps make the slide creation process a bit easier, and provides a glimpse into how helpful the Assistants API can be!

Here we have a template to create a Title Slide. The template below was created by uploading the image of a desirable title slide to GPT-V, and asking for the `python-pptx` code to create that template. The inputs to the template are the image_path, title_text, and subtitle_text.

In [ ]:
submit_message(assistant.id,thread,f"Use the included code template to create a PPTX slide that follows the template format, but uses the image, company name/title, and document name/subtitle included:\
{title_template}. IMPORTANT: Use the image file included in this message as the image_path image in this first slide, and use the Company Name {title_text} as the title_text variable, and \
  use the subtitle_text {subtitle_text} a the subtitle_text variable. \
    NEST, create a SECOND slide using the following code template: {data_vis_template} to create a PPTX slide that follows the template format, but uses the company name/title, and document name/subtitle included:\
{data_vis_template}. IMPORTANT: Use the line plot image, that is the second attached image in this message, that you created earlier in the thread as the data_vis_img image, and use the data visualization title that you created earlier for the variable title_text, and\
  the bullet points of insights you created earlier for the bullet_points variable. Output these TWO SLIDES as a .pptx file. Make sure the output is two slides, with each slide matching the respective template given in this message.",
              file_ids=[dalle_file.id, plot_file.id]
)


In [ ]:
#May take 1-3 mins
while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("Successfully retrieved pptx_id:", pptx_id)  # Add this line
        break
    except Exception as e:  # Catch the exception and print it
        print("Assistant still working on PPTX...")
        time.sleep(10)


In [ ]:
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file= client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("data/created_slides.pptx", "wb") as f:
    f.write(file_obj.getbuffer())


In [ ]:
![Title slide](data/title_slide.png)


In [ ]:
![Data slide](data/data_viz_slide.png)
